In [3]:
!pip install vtk

KeyboardInterrupt: 

In [1]:

import numpy as np
import torch
from torch import nn, optim
import torch.optim as optim
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import vtk
from vtk import *
from vtk.util.numpy_support import vtk_to_numpy
import random
import os
import sys
import time
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Device running:', device)

class SineLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        # self.enable_dropout = enable_dropout
        # self.dropout_prob = dropout_prob
        self.in_features = in_features
        # if enable_dropout:
        #     if not self.is_first:
        #         self.dropout = nn.Dropout(dropout_prob)
        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features,
                                             1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0,
                                             np.sqrt(6 / self.in_features) / self.omega_0)


    def forward(self, x):
        x = self.linear(x)
        # if self.enable_dropout:
        #     if not self.is_first:
        #         x = self.dropout(x)
        return torch.sin(self.omega_0 * x)

class ResidualSineLayer(nn.Module):
    def __init__(self, features, bias=True, ave_first=False, ave_second=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        # self.enable_dropout = enable_dropout
        # self.dropout_prob = dropout_prob
        self.features = features
        # if enable_dropout:
        #     self.dropout_1 = nn.Dropout(dropout_prob)
        self.linear_1 = nn.Linear(features, features, bias=bias)
        self.linear_2 = nn.Linear(features, features, bias=bias)
        self.weight_1 = .5 if ave_first else 1
        self.weight_2 = .5 if ave_second else 1

        self.init_weights()


    def init_weights(self):
        with torch.no_grad():
            self.linear_1.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)
            self.linear_2.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)

    def forward(self, input):
        linear_1 = self.linear_1(self.weight_1*input)
        # if self.enable_dropout:
        #     linear_1 = self.dropout_1(linear_1)
        sine_1 = torch.sin(self.omega_0 * linear_1)
        sine_2 = torch.sin(self.omega_0 * self.linear_2(sine_1))
        return self.weight_2*(input+sine_2)

class MyResidualSirenNet(nn.Module):
    def __init__(self, obj):
        super(MyResidualSirenNet, self).__init__()
        # self.enable_dropout = obj['enable_dropout']
        # self.dropout_prob = obj['dropout_prob']
        self.Omega_0=30
        self.n_layers = obj['n_layers']
        self.input_dim = obj['dim']
        self.output_dim = obj['total_vars']
        self.neurons_per_layer = obj['n_neurons']
        self.layers = [self.input_dim]
        for i in range(self.n_layers-1):
            self.layers.append(self.neurons_per_layer)
        self.layers.append(self.output_dim)
        self.net_layers = nn.ModuleList()
        for idx in np.arange(self.n_layers):
            layer_in = self.layers[idx]
            layer_out = self.layers[idx+1]
            ## if not the final layer
            if idx != self.n_layers-1:
                ## if first layer
                if idx==0:
                    self.net_layers.append(SineLayer(layer_in,layer_out,bias=True,is_first=idx==0))
                ## if an intermdeiate layer
                else:
                    self.net_layers.append(ResidualSineLayer(layer_in,bias=True,ave_first=idx>1,ave_second=idx==(self.n_layers-2)))
            ## if final layer
            else:
                final_linear = nn.Linear(layer_in,layer_out)
                ## initialize weights for the final layer
                with torch.no_grad():
                    final_linear.weight.uniform_(-np.sqrt(6 / (layer_in)) / self.Omega_0, np.sqrt(6 / (layer_in)) / self.Omega_0)
                self.net_layers.append(final_linear)

    def forward(self,x):
        for net_layer in self.net_layers:
            x = net_layer(x)
        return x

def size_of_network(n_layers, n_neurons, d_in, d_out, is_residual = True):
    # Adding input layer
    layers = [d_in]
    # layers = [3]

    # Adding hidden layers
    layers.extend([n_neurons]*n_layers)
    # layers = [3, 5, 5, 5]

    # Adding output layer
    layers.append(d_out)
    # layers = [3, 5, 5, 5, 1]

    # Number of steps
    n_layers = len(layers)-1
    # n_layers = 5 - 1 = 4

    n_params = 0

    # np.arange(4) = [0, 1, 2, 3]
    for ndx in np.arange(n_layers):

        # number of neurons in below layer
        layer_in = layers[ndx]

        # number of neurons in above layer
        layer_out = layers[ndx+1]

        # max number of neurons in both the layer
        og_layer_in = max(layer_in,layer_out)

        # if lower layer is the input layer
        # or the upper layer is the output layer
        if ndx==0 or ndx==(n_layers-1):
            # Adding weight corresponding to every neuron for every input neuron
            # Adding bias for every neuron in the upper layer
            n_params += ((layer_in+1)*layer_out)

        else:

            # If the layer is residual then proceed as follows as there will be more weights if residual layer is included
            if is_residual:
                # doubt in the following two lines
                n_params += (layer_in*og_layer_in)+og_layer_in
                n_params += (og_layer_in*layer_out)+layer_out

            # if the layer is non residual then simply add number of weights and biases as follows
            else:
                n_params += ((layer_in+1)*layer_out)
            #
        #
    #

    return n_params

def compute_PSNR(arrgt,arr_recon):
    diff = arrgt - arr_recon
    sqd_max_diff = (np.max(arrgt)-np.min(arrgt))**2
    snr = 10*np.log10(sqd_max_diff/np.mean(diff**2))
    return snr

def findMultiVariatePSNR(var_name, total_vars, actual, pred):
    # print('Printing PSNR')
    tot = 0
    psnr_list = []
    for j in range(total_vars):
        psnr = compute_PSNR(actual[:,j], pred[:,j])
        psnr_list.append(psnr)
        tot += psnr
        print(var_name[j], ' PSNR:', psnr)
    avg_psnr = tot/total_vars
    print('\nAverage psnr : ', avg_psnr)
     #this function is calculating the psnr of final epoch (or whenever it is called) of each variable and then averaging it
     #Thus individual epochs psnr is not calculated

    return psnr_list, avg_psnr

def compute_rmse(actual, predicted):
    mse = np.mean((actual - predicted) ** 2)
    return np.sqrt(mse)

def denormalizeValue(total_vars, to, ref):
    to_arr = np.array(to)
    for i in range(total_vars):
        min_data = np.min(ref[:, i])
        max_data = np.max(ref[:, i])
        to_arr[:, i] = (((to[:, i] * 0.5) + 0.5) * (max_data - min_data)) + min_data
    return to_arr

Device running: cuda


In [ ]:
from argparse import Namespace

# Parameters (simulating argparse in a Jupyter Notebook)
args = Namespace(
    n_neurons=150,
    n_layers=6,
    epochs=200,  # Required argument: Set the number of epochs
    batchsize=512,
    lr=0.00005,
    no_decay=False,
    decay_rate=0.8,
    decay_at_interval=True,
    decay_interval=15,
    datapath='/content/new_gmm_teardrop_week_9.vti',  # Required: Set the path to your data
    outpath='./models/',
    exp_path='../logs/',
    modified_data_path='./data/',
    dataset_name='3d_data',  # Required: Set the dataset name
    vti_name='new_Mine_teardrop_gmm_predicted_week9_150_three_models.vti',  # Required: Name of the dataset
    vti_path='./data/'
)

print(args, end='\n\n')

# Assigning parameters to variables
LR = args.lr
BATCH_SIZE = args.batchsize
decay_rate = args.decay_rate
decay_at_equal_interval = args.decay_at_interval

decay = not args.no_decay
MAX_EPOCH = args.epochs

n_neurons = args.n_neurons
n_layers = args.n_layers + 2
decay_interval = args.decay_interval
outpath = args.outpath
exp_path = args.exp_path
datapath = args.datapath
modified_data_path = args.modified_data_path
dataset_name = args.dataset_name
vti_name = args.vti_name
vti_path = args.vti_path

# Displaying the final configuration
print(f"Learning Rate: {LR}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Decay Rate: {decay_rate}")
print(f"Max Epochs: {MAX_EPOCH}")
print(f"Number of Neurons per Layer: {n_neurons}")
print(f"Number of Layers (including input/output): {n_layers}")
print(f"Data Path: {datapath}")
print(f"Output Path: {outpath}")
print(f"Dataset Name: {dataset_name}")
print(f"Vti Name: {vti_name}")

#datasetup
reader=vtk.vtkXMLImageDataReader()
reader.SetFileName(datapath)
reader.update()
image_data=reader.GetOutput()
point_data=image_data.GetPointData()
no_of_arrays=point_data.GetNumberOfArrays()
variables=[]
for i in range(no_of_arrays):
  array=point_data.GetArrayName(i)
  variables.append(array)
variables

origin=image_data.GetOrigin()
dim=image_data.GetDimensions()
spacing=image_data.GetSpacing()
num_points=dim[0]*dim[1]*dim[2]
print(num_points)

data=[]
for i in variables:
  vtk_array=point_data.GetArray(i)
  arr=vtk_to_numpy(vtk_array)
  data.append(arr)
print(data)

def scale(arr):
   min = np.min(arr)
   max = np.max(arr)
   arr = (arr - min) / (max - min)   # Scale to [0, 1]
   arr = 2 * (arr - 0.5)             # Scale to [-1, 1]
   return arr

x = scale(np.array(range(dim[0])))
y = scale(np.array(range(dim[1])))
z = scale(np.array(range(dim[2])))

print(np.min(x), np.max(x))

# Scaled coordinates already in x, y, z (each 1D, scaled to [-1, 1])
loc = []
for i in z:
    for j in y:
        for k in x:
            loc.append([k, j, i])  # [x, y, z] order
loc = np.array(loc)
print(loc.shape)  #

n_pts = image_data.GetNumberOfPoints()
scaled_data = data.copy()
real_data=data.copy()
for i in range(9):  # 0-5: means and stds
    min_val = np.min(scaled_data[i])
    max_val = np.max(scaled_data[i])
    scaled_data[i] = (scaled_data[i] - min_val) / (max_val - min_val)  # [0,1]
    scaled_data[i] = 2 * (scaled_data[i] - 0.5)  # [-1,1]
scaled_data = np.array(scaled_data).T
scaled_data.shape

t_data=torch.from_numpy(scaled_data)
t_loc=torch.from_numpy(loc)

print('Dataset Name:', dataset_name)
print('Total Variables:', no_of_arrays)
print('Variables Name:', variables, end="\n\n")
print('Total Points in Data:', num_points)
print('Dimension of the Dataset:', dim)
print('Number of Dimensions:',len(dim))
print('Coordinate Tensor Shape:', t_loc.shape)
print('Scalar Values Tensor Shape:', t_data.shape)

print('\n###### Data setup is complete, now starting training ######\n')
import random
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
train_dataloader = DataLoader(
    TensorDataset(t_loc, t_data),
    batch_size=BATCH_SIZE,
    pin_memory=True,
    shuffle=True,
    num_workers=4
)

obj = {
    'total_vars': no_of_arrays,
    'dim': len(dim),
    'n_neurons': n_neurons,
    'n_layers': n_layers
}
print(obj)
model=MyResidualSirenNet(obj).to(device)
print(model)

optimizer = optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))
print(optimizer)

criterion = nn.MSELoss()
print(criterion)

group_size = 5000
univariate=None

output_groups = {
    'means': [0, 1, 2],
    'stds': [3, 4, 5],
    'weights': [6, 7, 8]
}

best_losses = {k: 1e8 for k in output_groups}
best_epochs = {k: -1 for k in output_groups}
models = {}
optimizers = {}
train_loss_logs = {k: [] for k in output_groups}

# Model constructor
def create_model():
    obj = {
        'total_vars': 3,
        'dim': 3,  # x, y, z
        'n_neurons': n_neurons,
        'n_layers': n_layers
    }
    return MyResidualSirenNet(obj).to(device)

# Initialize output path
if not os.path.exists(outpath):
    os.makedirs(outpath)


In [ ]:
# Train separate models for each component group
for name, indices in output_groups.items():
    print(f"\nTraining model for: {name.upper()}")
    model = create_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    train_loss_list = train_loss_logs[name]
    if(name=='means'):
        MAX_EPOCH=200
    else:
        MAX_EPOCH=200

    for epoch in tqdm(range(MAX_EPOCH)):
        model.train()
        temp_loss_list = []
        start = time.time()

        for X_train, y_train in train_dataloader:
            X_train = X_train.float().to(device)
            y_train = y_train.float().to(device)

            y_target = y_train[:, indices]

            optimizer.zero_grad()
            predictions = model(X_train).squeeze()
            loss = criterion(predictions, y_target)
            loss.backward()
            optimizer.step()

            temp_loss_list.append(loss.detach().cpu().numpy())

        epoch_loss = np.mean(temp_loss_list)
        train_loss_list.append(epoch_loss)

        # Learning rate decay
        if decay:
            if decay_at_equal_interval and epoch >= decay_interval and epoch % decay_interval == 0:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= decay_rate
            elif epoch > 0 and epoch_loss > train_loss_list[-2]:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= decay_rate

        # Save best model
        if epoch_loss < best_losses[name]:
            best_losses[name] = epoch_loss
            best_epochs[name] = epoch + 1
            torch.save(
                {"epoch": epoch + 1, "model_state_dict": model.state_dict()},
                os.path.join(outpath, f'{name}_best_model.pth')
            )

        # Log progress
        end = time.time()
        print(f"Epoch {epoch+1}/{MAX_EPOCH} | {name.upper()} Loss: {epoch_loss:.6f} | "
              f"Time: {round(end - start, 2)}s | LR: {optimizer.param_groups[0]['lr']:.6e}")

In [ ]:
# Final summary
for name in output_groups:
    print(f"\nBest {name.upper()} model at epoch {best_epochs[name]} with loss {best_losses[name]:.6f}")

# Load and prepare models
output_groups = {
    'means': [0, 1, 2],
    'stds': [3, 4, 5],
    'weights': [6, 7, 8]
}

prediction_dict = {}

# Define model creation for loading
def create_model():
    obj = {
        'total_vars': 3,
        'dim': 3,
        'n_neurons': n_neurons,
        'n_layers': n_layers
    }
    return MyResidualSirenNet(obj).to(device)

# Inference loop
with torch.no_grad():
    for name in output_groups:
        print(f"\nRunning inference for: {name.upper()}")

        # Load best model
        model = create_model()
        model_path = os.path.join(outpath, f"{name}_best_model.pth")
        model.load_state_dict(torch.load(model_path)['model_state_dict'])
        model.eval()

        predictions = []

        # Batch-wise prediction on coordinates
        for i in range(0, t_loc.shape[0], group_size):
            coords = t_loc[i:min(i + group_size, t_loc.shape[0])].to(device).float()
            output = model(coords).to('cpu')
            predictions.append(output)

        # Concatenate predictions
        prediction_dict[name] = torch.cat(predictions, dim=0).numpy()

# Combine all into final output
means_pred = prediction_dict['means']    # shape (N, 3)
stds_pred = prediction_dict['stds']      # shape (N, 3)
weights_pred = prediction_dict['weights']# shape (N, 3)

# Final combined prediction: (N, 9)
n_predictions = np.concatenate([means_pred, stds_pred, weights_pred], axis=1)

# Evaluation
total_vars = 9
findMultiVariatePSNR(variables, total_vars,scaled_data, n_predictions)
rmse = compute_rmse(scaled_data, n_predictions)
print("Final RMSE:", rmse)



In [ ]:
def makeVTI(data, val, n_predictions, n_pts, total_vars, var_name, dim, isMaskPresent, mask_arr, vti_path, vti_name, normalizedVersion = False):
    nn_predictions = denormalizeValue(total_vars, n_predictions, val) if not normalizedVersion else n_predictions
    writer = vtkXMLImageDataWriter()
    writer.SetFileName(vti_path + vti_name)
    img = vtkImageData()
    img.CopyStructure(data)
    if not isMaskPresent:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            arr = vtkFloatArray()
            for j in range(n_pts):
                arr.InsertNextValue(temp[j])
            arr.SetName(f)
            img.GetPointData().AddArray(arr)
        # print(img)
        writer.SetInputData(img)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')
    else:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            idx = 0
            arr = vtkFloatArray()
            for j in range(n_pts):
                if(mask_arr[j] == 1):
                    arr.InsertNextValue(temp[idx])
                    idx += 1
                else:
                    arr.InsertNextValue(0.0)
            arr.SetName('p_' + f)
            data.GetPointData().AddArray(arr)
        # print(data)
        writer.SetInputData(data)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')

In [ ]:
def denormalizeValue(total_vars, to, ref):
    to_arr = np.array(to)
    for i in range(total_vars):
        min_data = np.min(ref[:, i])
        max_data = np.max(ref[:, i])
        to_arr[:, i] = (((to[:, i] * 0.5) + 0.5) * (max_data - min_data)) + min_data
    return to_arr

In [ ]:
def makeVTI(data, val, n_predictions, n_pts, total_vars, var_name, dim, isMaskPresent, mask_arr, vti_path, vti_name, normalizedVersion = False):
    nn_predictions = denormalizeValue(total_vars, n_predictions, val) if not normalizedVersion else n_predictions
    writer = vtkXMLImageDataWriter()
    writer.SetFileName(vti_path + vti_name)
    img = vtkImageData()
    img.CopyStructure(data)
    if not isMaskPresent:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            arr = vtkFloatArray()
            for j in range(n_pts):
                arr.InsertNextValue(temp[j])
            arr.SetName(f)
            img.GetPointData().AddArray(arr)
        # print(img)
        writer.SetInputData(img)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')
    else:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            idx = 0
            arr = vtkFloatArray()
            for j in range(n_pts):
                if(mask_arr[j] == 1):
                    arr.InsertNextValue(temp[idx])
                    idx += 1
                else:
                    arr.InsertNextValue(0.0)
            arr.SetName('p_' + f)
            data.GetPointData().AddArray(arr)
        # print(data)
        writer.SetInputData(data)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')

In [ ]:
def getImageData(actual_img, val, n_pts, var_name, isMaskPresent, mask_arr):
    img = vtkImageData()
    img.CopyStructure(actual_img)
    # if isMaskPresent:
    #     img.DeepCopy(actual_img)
    # img.SetDimensions(dim)
    # img.SetOrigin(actual_img.GetOrigin())
    # img.SetSpacing(actual_img.GetSpacing())
    if not isMaskPresent:
        f = var_name
        data = val
        arr = vtkFloatArray()
        for j in range(n_pts):
            arr.InsertNextValue(data[j])
        arr.SetName(f)
        img.GetPointData().SetScalars(arr)
    else:
        f = var_name
        data = val
        idx = 0
        arr = vtkFloatArray()
        for j in range(n_pts):
            if(mask_arr[j] == 1):
                arr.InsertNextValue(data[idx])
                idx += 1
            else:
                arr.InsertNextValue(0.0)
        arr.SetName(f)
        img.GetPointData().SetScalars(arr)
    return img


In [ ]:
# vti saving path
vti_path = args.vti_path
if not os.path.exists(vti_path):
    os.makedirs(vti_path)
# vti name
isMaskPresent=False
mask_arr=[]
vti_name = args.vti_name
real_data=np.array(real_data)
makeVTI(image_data, real_data, n_predictions, n_pts, total_vars, variables, dim, isMaskPresent, mask_arr, vti_path, vti_name)

Vti File written successfully at ./data/new_Mine_teardrop_gmm_predicted_week9_150_three_models.vti
